# Path-integral Monte Carlo for the 1d oscillator
Jan Gukelberger, Andreas Hehn, Georg Winkler, Dominik Gresch (2011-2017)

In [ ]:
from __future__ import division, print_function

import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt
from sys import stdout
%matplotlib inline
plt.rcParams['figure.figsize'] = 16, 9

In [ ]:
# hbar = m = 1
w = 1.0
# seed random number generator once at program start
rnd.seed(42)

### Class for storing world-line configuration and doing updates/measurements

A note on kinetic energy: it was shown that calculating the kinetic energy of a configuration as $\langle KE\rangle=\frac{1}{2M}\sum_{i=0}^{M-1}\left(\frac{x_{j+1}-x_j}{\Delta\tau}\right)^2$ is unstable when the number of timeslices is large due to the growth of the variance of the estimator. A numerically preferred form is $\langle KE\rangle=\frac{1}{2}\langle x\frac{dV}{dx}\rangle$, obtained from the Virial theorem.

Reference: https://aip.scitation.org/doi/10.1063/1.442815

In [ ]:
def V_oscillator(x):
    return w ** 2 * x ** 2 / 2.

def V_Higgs(x, eta):
    return (x ** 2 - eta ** 2) ** 2

def dVdx_oscillator(x):
    return w ** 2 * x

def dVdx_Higgs(x, eta):
    return 4 * x  * (x ** 2 - eta ** 2)


class Config:
    """PIMC configuration: world-line for one particle"""

    def __init__(self, beta, numslices, V, dVdx):
        self._beta = beta
        self._numslices = numslices
        self._tau = self._beta / self._numslices    # \Delta\tau=\beta/M
        self._config = rnd.uniform(-1., 1., numslices)    # initial configuration
        self.V = V
        self.dVdx = dVdx

    def potential_energy(self):
        """Return the potential energy of a configuration X"""
        # implement here
        return 

    def kinetic_energy(self):
        """Return the kinetic energy of a configuration X"""
        # implement here
        return 
        
    def position_histogram(self, bins, value_range):
        """Return histogram of positions in all time slices"""
        return np.histogram(self._config, bins, range=value_range)[0]

    def update(self, max_displacement):
        """Metropolis algorithm local configuration update"""
        j = # pick a random time slice 
        new_position_j = # propose a new position where \Delta x\in[-max_displacement, max_displacement]

        # periodic boundary conditions 
        jp1 = (j + 1) % self._config.size

        acceptance_ratio = # as given in the lecture

        if acceptance_ratio >= 1 or rnd.uniform() < acceptance_ratio:
            self._config[j] = new_position_j
            return True
        else:
            return False

    def sweep(self, max_displacement):
        """One sweep of Metropolis local updates (i.e. self._slices update proposals)"""
        accepted_proposals = 0
        for l in range(self._config.size):
            accepted_proposals += self.update(max_displacement)
        return accepted_proposals / self._numslices    # to calculate acceptance rate later

### Autocorrelation analysis

This is again binning analysis, the same as the past 2 exercises.

In [ ]:
def stats(samples, n_levels=9):
    """
    Perform a binning analysis over samples and return an array of the error estimates at each binning level.
    
    """
    bins = np.array(samples)
    errors = np.zeros(n_levels + 1)
    errors[0] = np.std(bins) / np.sqrt(len(bins) - 1)
    for k in range(n_levels):
        bins = np.array([(bins[2*i]+bins[2*i+1])/2. for i in range(len(bins)//2)])
        errors[k+1] = np.std(bins) / np.sqrt(len(bins) - 1)
    # calculate autocorrelation time
    tau = 0.5*(errors[-1]**2/np.std(samples)**2*(len(samples)-1.)-1.)
    
    return np.mean(samples), errors[-1], tau

### PIMC simulation and measuring energies

Here you implement the core part of the simulation and the autocorrelation analysis.

In [ ]:
# simulation parameters
beta = 1.
P = 10
max_displacement = .5
# parameters for wave function measurements (x histogram)
histo_range = (-4.0, 4.0)
histo_bins = 100
histo_samples = 64

def simulate(config, steps, thermal_steps):
    # initialize configuration and observables
    potential_energy = np.empty(steps, dtype=float)
    kinetic_energy = np.empty(steps, dtype=float)
    position_histogram = np.zeros((histo_samples, histo_bins))
    acc_rate = 0.

    # thermalize configuration
    print('Thermalization (' + str(thermal_steps) + ' sweeps)...')
    # implement here

    # simulation: measures after each update sweep
    print('Simulation (' + str(steps) + ' sweeps)')
    
    for i in range(steps):
        # implement here; track acceptance rate, PE, KE and wavefunction (position_histogram)

        # Progress marker: one . for each percent
        if i % (steps // 100) == 0:
            stdout.write('.')
            stdout.flush()

    # If the acceptance rate is not somewhere around 0.5, max_displacement needs to be tuned.
    acc_rate /= steps
    print('\nAcceptance rate = ' + str(acc_rate))
    
    return potential_energy, kinetic_energy, position_histogram


def autocorrelation_analysis(potential_energy, kinetic_energy, steps):
    # Evaluate results
    # get the mean, error and autocorrelation time for PE, KE and total energy
    pot, pot_error, pot_autocorr = ...
    kin, kin_error, kin_autocorr = ...
    etot, etot_error, etot_autocorr = ...

    # running mean
    pot_series = np.cumsum(potential_energy) / np.arange(1, steps + 1)
    kin_series = np.cumsum(kinetic_energy) / np.arange(1, steps + 1)

    print("Potential Energy = " + str(pot) + " +/- " +
          str(pot_error) + "\tCorrelation time: " + str(pot_autocorr))
    print("Kinetic Energy   = " + str(kin) + " +/- " +
          str(kin_error) + "\tCorrelation time: " + str(kin_autocorr))
    print("Total Energy     = " + str(etot) + " +/- " + 
          str(etot_error) + "\tCorrelation time: " + str(etot_autocorr))
    
    return pot, kin, pot_series, kin_series


#### Plotting the energy

Take care to normalise the histogram!

In [ ]:
def plot_energy(potential_energy, kinetic_energy, pot, kin, pot_series, kin_series, position_histogram, steps, ylim=[0.1, 0.7]):
    # Plot raw samples
    plt.figure()
    plt.title('Potential Energy Samples')
    plt.xlabel('MC step')
    plt.ylabel('potential energy')
    plt.plot(potential_energy, label='$V_i$')
    plt.plot([0, steps - 1], [pot, pot], label='$\\bar{V}$')
    plt.xlim([0, 10000])
    plt.legend()

    # Plot raw samples
    plt.figure()
    plt.title('Kinetic Energy Samples')
    plt.xlabel('MC step')
    plt.ylabel('kinetic energy')
    plt.plot(range(steps),kinetic_energy,label='$T_i$')
    plt.plot([0,steps-1],[kin,kin],label='$\\bar{T}$')
    plt.xlim([0, 10000])
    plt.legend()

    # Plot running mean
    plt.figure()
    plt.title('Time series for energy observables')
    plt.xlabel('MC steps')
    plt.ylabel('energy')
    plt.plot(range(steps), pot_series, label='$\\bar{V}_i$')
    plt.plot([0, steps - 1], [pot, pot], label='$\\bar{V}$')
    plt.plot(range(steps), kin_series, label='$\\bar{T}_i$')
    plt.plot([0, steps - 1], [kin, kin], label='$\\bar{T}$')
    plt.ylim(ylim)
    plt.legend()

    # Normalize histogram and calculate error bars:
    # We did not collect a complete time series, but a fixed number of bins.
    # This works as long as the size of each bin [steps/histo_samples] >>
    # [autocorrelation time]
    position_histogram /= np.sum(position_histogram,
                                 axis=1).reshape((histo_samples, 1))
    histomean = np.mean(position_histogram, axis=0)
    histoerr = np.std(position_histogram, axis=0) / np.sqrt(histo_samples - 1)

    # Plot wave function
    plt.figure()
    plt.title('Wave function')
    plt.xlabel('x')
    plt.ylabel("$|\\psi|^2$")
    binwidth = (histo_range[1] - histo_range[0]) / histo_bins
    plt.errorbar(np.linspace(histo_range[
                 0] + binwidth / 2, histo_range[1] - binwidth / 2, histo_bins), histomean, histoerr)
    plt.show()
    
    return None


## 1. Harmonic oscillator

#### PIMC simulation

In [ ]:
step = 100000
thermal_step = 20000  
c = Config(beta, P, V_oscillator, dVdx_oscillator)
pe, ke, xs = simulate(c, step, thermal_step)

In [ ]:
pe_mean, ke_mean, pe_running, ke_running = autocorrelation_analysis(pe, ke, step)
print('Exact result   E = ' + str(.5 * w / np.tanh(.5 * w * beta)))

In [ ]:
plot_energy(pe, ke, pe_mean, ke_mean, pe_running, ke_running, xs, step)

## 2. Higgs potential

Let us first plot the potential at various values of $\eta$. There are two minima at $\pm\eta$.

In [ ]:
x = np.linspace(-4., 4., 100)
eta_vals = [1., 2., 3.]
plt.figure(figsize=(6,4))
for eta in eta_vals:
    plt.plot(x, V_Higgs(x, eta), label='$\eta=$'+str(eta))
plt.xlabel('x')
plt.ylabel('V')
plt.legend()
plt.show()

#### PIMC simulation

In [ ]:
step = 100000
thermal_step = 20000 
eta = [1., 3.]
lims = [[0.2, 0.9], [1.2, 3.]]

for e, lim in zip(eta, lims):
    c = Config(beta, P, lambda x: V_Higgs(x, e), lambda x: dVdx_Higgs(x, e))
    pe, ke, xs = simulate(c, step, thermal_step)
    pe_mean, ke_mean, pe_running, ke_running = autocorrelation_analysis(pe, ke, step)
    plot_energy(pe, ke, pe_mean, ke_mean, pe_running, ke_running, xs, step, lim)
    